제 1유형

1. 각 연도 및 성별의 총 대출액의 절댓값 차이를 구하고, 절댓값 차이가 가장 큰 지역 코드를 구하시오

2. 각 연도별 최대 검거율을 가진 범죄유형을 찾아서 해당 연도 및 유형의 검거건수들의 총합을 구하시오.

3. 제시된 문제를 순서대로 풀고, 해답을 제시하시오
    3.1. 평균만족도: 결측치는 평균만족도 컬럼의 전체 평균으로 채우시오.
    3.2. 근속연수: 결측치는 각 부서와 등급별 평균 근속연수로 채우시오.
    3.3. A: 부서가 'HR'이고 등급이 'A'인 사람들의 평균 근속연수를 계산하시오.
    3.4. B: 부서가 'Sales'이고 등급이 'B'인 사람들의 평균 교육참가횟수를 계산하시오.
    3.5. A와 B를 더한 값을 구하시오.

In [ ]:
import pandas as pd
df = pd.read_csv('9_1_1.csv')
#print(df.head())

q1 = df.groupby(['year', 'gender', '지역코드'])[['금액1', '금액2']].sum().reset_index()
q1['diff'] = abs(q1['금액1'] - q1['금액2'])
max_diff = q1.loc[q1['diff'].idxmax()]
print(max_diff['지역코드'])


167


In [ ]:
import pandas as pd
df = pd.read_csv('9_1_2.csv')
print(df.head())

df_occur = df[df['구분']=='발생건수'].set_index('연도').drop(columns='구분')
df_arrest = df[df['구분']=='검거건수'].set_index('연도').drop(columns='구분')

ratio = df_occur / df_arrest


     연도    구분  범죄유형1  범죄유형2  범죄유형3  범죄유형4  범죄유형5  범죄유형6  범죄유형7  범죄유형8  범죄유형9  \
0  2000  발생건수    302    635    470    306    271    900    220    814    321   
1  2000  검거건수    302    430    470    187    271    199    220    230    321   
2  2001  발생건수    969    543    691    613    585    391    476    360    659   
3  2001  검거건수    121    352    691    613    574    158    476    360    575   
4  2002  발생건수    982    389    886    762    766    443    704    330    684   

   범죄유형10  
0     666  
1     408  
2     513  
3     513  
4     846  


제 2유형

훈련 데이터로 학습한 모델을 테스트 데이터에 적용하여 예측한 결과를 제출하시오.
(Target: 농업유형(다중분류, {0,1,2}))
% 제출 형식은 ID, pred 두 칼럼만 존재해야 한다.(평가 지표: Marco F1 Score )